\begin{center}
Gabe Morris
\end{center}

In [1]:
# Notebook Preamble
%config ZMQInteractiveShell.ast_node_interactivity = 'all'
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

t, s = sp.symbols('t s')

plt.style.use('maroon_ipynb.mplstyle')

\pagebreak
\tableofcontents
\pagebreak

\begin{center}
\begin{tabular}{c c c}
ME 6643 & Homework 9 & Gabe Morris \\
& & gnm54
\end{tabular}
\end{center}

# Problem 10.4
## Given

\begin{center}
\includegraphics{images/fig1}
\end{center}

## Find
Derive the output $C(s)$, error $E(s)$, and actuator $M(s)$ equations for the diagram above and obtain the characteristic polynomial.

## Solution

In [2]:
# Use algebra to determine the functions
K = sp.Symbol('K')
R, E, F, M, D, C, T = [sp.Function(i)(s) for i in ['R', 'E', 'F', 'M', 'D', 'C', 'T']]
eq1 = sp.Eq(E, R - C)
eq1
eq2 = sp.Eq(F, E*K)
eq2
eq3 = sp.Eq(M, F*4/(3*s + 1))
eq3
eq4 = sp.Eq(C, (M - D)*6/(15*s + 2))
eq4

Eq(E(s), -C(s) + R(s))

Eq(F(s), K*E(s))

Eq(M(s), 4*F(s)/(3*s + 1))

Eq(C(s), (-6*D(s) + 6*M(s))/(15*s + 2))

The characteristic polynomial may be determined by solving for $\frac{C(s)}{R(s)}$.

In [3]:
subs = eq4.subs([
    (M, eq3.rhs),
    (F, eq2.rhs),
    (E, eq1.rhs),
    (C, T*R)
])
sol = sp.solve(subs, T)[0]
sp.Eq(T, sol)

Eq(T(s), 6*(4*K*R(s) - 3*s*D(s) - D(s))/((24*K + 45*s**2 + 21*s + 2)*R(s)))

$\frac{C(s)}{R(s)}$ may be obtained by setting $D(s)$ equal to 0.

In [4]:
sp.Eq(C/R, sol.subs(D, 0))

Eq(C(s)/R(s), 24*K/(24*K + 45*s**2 + 21*s + 2))

It is not necessary to try and rewrite $E(s)$, $M(s)$, and $F(s)$ in any other forms. The denominator of the $\frac{C(s)}{R(s)}$ function above is the characteristic polynomial,

$$45s^2+21s+2+24K$$